# Comparing the effects of different longwave radiation parameterization on simulated snow

## Introduction
In this exercise we will explore how variability and uncertainty in longwave radiation affects simulations of snow. Longwave radiation, or thermal radiation, is emitted from objects and depends on the object's "emissivity" and temperature. The longwave radiation can be calculated as

### $$ LW = \epsilon \sigma T^4 $$

where $\epsilon$ is the emissivity, $\sigma$ is the [Stephan-Boltzmann constant](https://en.wikipedia.org/wiki/Stefan%E2%80%93Boltzmann_constant), and $T$ is the temperature (in Kelvin). Because longwave radiation is rarely measured, many models require estimations to close the energy balance. The FluxNet community maintains a large number of the sites where longwave measurements are taken across the world.

#### **Locations of FluxNet towers throughout the world**
![locations of fluxnet towers throughout the world](./assets/fluxnet.png)

As you can see, there are only a couple hundred sites throughout the world and are particularly concentrated in North America and Europe. While other locations and consortiums for measuring longwave exist (such as a handful of SnoTel sites), they are still _very_ sparse throughout the world, and even through the United States. So, if we want to be able to model basins without observations we will need to estimate the longwave radiation. This mostly amounts to estimation of the emissivity, as temperature can be more easily taken from measurement, reanalysis, or simulated  (somewhat) reliably. There are a multitude of methods for estimating this, of which we will explore a few.

## Study sites 

To do so, we have provided SUMMA setups for 3 sites with different hydroclimates. These sites were chosen because they do have longwave radiation measurements, and allow us to compare the simulated snowpack with the different estimation methods. These sites are Dana Meadows in Yosemite National park, Reynolds Mountain East (part of the [Reynolds Creek Experimental Watershed](https://czo-archive.criticalzone.org/reynolds/infrastructure/field-areas-reynolds/) in southwestern Idaho, and Col de Porte in the Chartreuse Mountains near Grenoble, France. The three site locations and their long term temperature records are shown below.

#### **Locations of study sites**
![locations of study sites](./assets/snow_sites.png)

We have put together three years of data for each of the sites which can be used to run the SUMMA hydrologic model. SUMMA takes longwave radiation as an input variable, so we will provide both the observed and estimated values to compare how they affect the snowpack evolution. As mentioned above, there are several methods for estimating longwave radiation. 

## Longwave radiation parameterizations
We have provided functionality for several of these, which we outline (with references) below:




### TVA
Heat and mass transfer between a water surface and the atmosphere. Tennessee Valley Authority, Norris, TN. Laboratory report no. 14. Water resources research report
no. 0-6803.

### $$ \epsilon = 0.74 + 0.0049 \cdot e $$

### Anderson
Anderson, E.R., 1954. Energy budget studies, water loss
investigations: lake Hefner studies. U.S. Geol. Surv. Prof. Pap. 269,
71–119 Available from U.S. Geological Survey, 807 National Center, Reston, VA 20192.

### $$ \epsilon = 0.68 + 0.036 \cdot \sqrt{e}$$

### Satterlund
Satterlund, D.R., 1979. An improved equation for estimating long-wave
radiation from the atmosphere. Water Resour. Res. 15 (6), 1649–1650,
 doi:10.1029/WR015i006p01649.
        
### $$ \epsilon = 1.08 (1 - exp(-e^{T/2016} )$$

### Prata

Prata, A.J., 1996. A new long-wave formula for estimating downward
clear-sky radiation at the surface. Q. J. R. Meteor. Soc. 122 (533),
1127–1151, doi:10.1002/qj.49712253306.

### $$ r = (46.4 \cdot (e/T)) $$
### $$ \epsilon = 1- (1+r) \cdot exp\left(-\sqrt{1.2 + 3\cdot r)}\right) $$

### DOKIA
We also provide one more parameterization, which has a slightly different formulation.

Clear sky reference: 

Dilley, A. C., and D. M. O<92>Brien (1998), Estimating downward clear sky
long-wave irradiance at the surface from screen temperature and precipitable
water, Q. J. R. Meteorol. Soc., 124, 1391<96> 1401.

Cloudy sky reference:

Kimball, B. A., S. B. Idso, and J. K. Aase (1982), A model of thermal
radiation from partly cloudy and overcast skies, Water Resour. Res., 18,
931<96> 936.     

### $$ L_{clr} = 59.38 + 113 (\frac{T}{273.16})^6 + 96.96\sqrt{w/25} $$

### $$ L_{cld} = \tau_8 \cdot c \cdot f_8 (T-11)^4$$
where
$$ \epsilon = 0.24 + 2.98 \cdot 10^{-6} VP^2 e^{3000/T} $$
$$ \tau = 1-\epsilon (1.4 - (0.4\epsilon)) $$
$$ f = -0.6723 + 0.624 \cdot 10^{-2} \cdot (T-11) - 0.914\cdot10^{-5} (T-11)^2 $$
and $w$ is precipitable water.

## Your assignment

As described above, we have provided implementations of each of these longwave parameters which you will compare. We have also provided the data and functions to get you started. To begin we import some standard librarires. The `utilities` library is where we have implemented the specific functionality for you to be able to modify the longwave radiation. Additionally, we have implemented simple cloud correction schemes, which further modify the longwave radiation when there is cloud cover. You can think of clouds as a blanket which insulate the land surface, increasing the longwave radiation. As you might have noticed above, this cloud cover correction is built into the DOKIA method. We will describe how to activate the cloud cover fraction later.

To get started, import the libraries below.

In [ ]:
import utilities
from matplotlib import pyplot as plt
import numpy as np
import xarray as xr
import pysumma as ps
import pandas as pd

## Site selection

To make things more computationally tractable, you will only have to select a single site for your assignment. You can set it below. The line beginning with a `!` is known as a "bang", which indicates a [`bash`](https://www.gnu.org/software/bash/) command. `bash` is a language for interacting with Linux environments, which HydroShare runs on. This command updates a configuration file so that you are ready to run the default simulation. Running the following cell will run the simulation with the observed longwave radiation, as well as open up the observed SWE for your site. If you do not see a `'Success'` print after running the following cell, something has gone wrong and you should ask for help!

In [ ]:
# Select one of 'dana', 'reynolds', or 'coldeport'
site = 'dana'
# This line is needed to set the file manager up
! cd ./data/{site}/ && ./install_local_setup.sh && cd -

# Where the SUMMA program lives
summa_exe = 'summa.exe'

# The main SUMMA configuration file is here
file_manager = f'./data/{site}/file_manager.txt'

# Open up the observation data here
obs = xr.open_dataset(f'./data/{site}/observations/{site}_obs.nc')

# Make a new pysumma Simulation object and run it 
# following running, make sure things went okay
sim_default = ps.Simulation(summa_exe, file_manager)
sim_default.run('local')
print(sim_default.status)

## Modifying the longwave

As we mentioned, we have implemented the various longwave parameterizations in the `utilities` library. We have also implemented a helper function that can get everything set up for you. This function is named `modify_longwave`. To see it's "function signature" (which defines the inputs and outputs of the function), go ahead and run the next cell.

In [ ]:
?utilities.modify_longwave

## Explaining the above

From the above function signature, you should see that the function takes in three inputs:

 - `parameterization`: The name of the longwave parameterization to use
 - `manager_path`: The path to a file manager for SUMMA, to use as a template
 - `do_cloud_correction`: Whether to apply a cloud correction to the longwave radiation
 
And a single output: A new file manager that you can use to make new `ps.Simulation` objects with the modified longwave radiation

## Your tasks:

- Choose one of the longwave parameterizations to run
- Run new simulations both with, and without cloud corrections (This will require two calls to `utilities.modify_longwave` as well as two new `ps.Simulation` objects.)
- Make plots of the longwave radiation (variable name is `LWRadAtm`) for the observed, with cloud correction, and without cloud correction
- Make plots of the simulated SWE for each of the simulations
- Comment on differences
- For bonus, consider looking at the variable `scalarSnowAlbedo`, `scalarSnowSublimation`, and `scalarLatHeatTotal`!
- For extra bonus, consider looking at multiple parameterizations!

In [ ]:
# Select one of 'dokia', 'anderson', 'satterlund', 'tva', 'prata'
parameterization = 'satterlund'

In [ ]:
lw_file_manager_nocloud = utilities.modify_longwave(parameterization, file_manager, do_cloud_correction=False)
sim_lw_nocloud = ps.Simulation(summa_exe, lw_file_manager_nocloud)
sim_lw_nocloud.run('local')
print(sim_lw_nocloud.status)

In [ ]:
lw_file_manager_cloud = utilities.modify_longwave(parameterization, file_manager, do_cloud_correction=True)
sim_lw_cloud = ps.Simulation(summa_exe, lw_file_manager_cloud)
sim_lw_cloud.run('local')
print(sim_lw_nocloud.status)

In [ ]:
(sim_lw_nocloud.output['LWRadAtm'].resample({'time': 'D'}).mean()
                                  .plot(color='orange', linewidth=2, label=f'clear sky {parameterization}'))
(sim_lw_cloud.output['LWRadAtm'].resample({'time': 'D'}).mean()
                                .plot(color='crimson', linewidth=2, label=f'cloud corrected {parameterization}'))

(sim_default.output['LWRadAtm'].resample({'time': 'D'}).mean()
                               .plot(color='blue', linewidth=2, label='observed longwave'))
plt.legend()

In [ ]:
sim_default.output

In [ ]:
obs['SWE'].plot(color='black', linewidth=2, label='observed SWE')
sim_default.output['scalarSWE'].plot(color='blue', linewidth=2, label='using obs LW')
sim_lw_nocloud.output['scalarSWE'].plot(color='orange', linewidth=2, label='clear sky LW')
sim_lw_cloud.output['scalarSWE'].plot(color='crimson', linewidth=2, label='cloud corrected LW')
plt.legend()

In [ ]:
sim_default.output['scalarSnowDepth'].plot(color='blue', linewidth=2, label='using obs LW')
sim_lw_nocloud.output['scalarSnowDepth'].plot(color='orange', linewidth=2, label='clear sky LW')
sim_lw_cloud.output['scalarSnowDepth'].plot(color='crimson', linewidth=2, label='cloud corrected LW')
plt.legend()